# Introduction

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("factbook.db")
cursor = conn.cursor()
q1 = "select * from sqlite_master where type='table';"
pd.read_sql_query(q1,conn)
cursor.execute(q1).fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'facts',
  'facts',
  47,
  'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "area_land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

# Overview of the Data

In [2]:
q2 = "select * from facts limit 5"
pd.read_sql_query(q2,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

In [3]:
q3 = "select min(population),max(population),min(population_growth),max(population_growth) from facts"
pd.read_sql_query(q3,conn)

,min(population),max(population),min(population_growth),max(population_growth)
0,0,7256490011,0.0,4.02


# Exploring Outliers

In [4]:
q4 = "select * from facts where population == 0"
pd.read_sql_query(q4,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [5]:
q5 = "select * from facts where population == (select max(population) from facts)"
pd.read_sql_query(q5,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


# Exploring Average Population and Area

In [6]:
q6 = "select AVG(population),AVG(area) from facts"
pd.read_sql_query(q6,conn)

,AVG(population),AVG(area)
0,6.209493e+07,555093.546185


# Finding Densely Populated Countries

In [7]:
q7 = "select * from facts where population > (select AVG(population) from facts) and area < (select AVG(area) from facts)"
pd.read_sql_query(q7,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,14,bg,Bangladesh,148460,130170,18290,168957745,1.60,21.14,5.61,0.46
1,65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
2,85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.00
3,138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
4,173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.80,0.00
5,185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
6,192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.30


# 20 Countries will add the most people to their population next year

In [8]:
q9 = """select * 
from facts 
where population != 0 and population_growth != 0
order by population * population_growth DESC limit 20"""
pd.read_sql_query(q9,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,261,xx,World,NaN,NaN,NaN,7256490011,1.08,18.60,7.80,NaN
1,77,in,India,3287263.0,2973193.0,314070.0,1251695584,1.22,19.55,7.32,0.04
2,37,ch,China,9596960.0,9326410.0,270550.0,1367485388,0.45,12.49,7.53,0.44
3,129,ni,Nigeria,923768.0,910768.0,13000.0,181562056,2.45,37.64,12.90,0.22
4,132,pk,Pakistan,796095.0,770875.0,25220.0,199085847,1.46,22.58,6.49,1.54
5,58,et,Ethiopia,1104300.0,NaN,104300.0,99465819,2.89,37.27,8.19,0.22
6,14,bg,Bangladesh,148460.0,130170.0,18290.0,168957745,1.60,21.14,5.61,0.46
7,186,us,United States,9826675.0,9161966.0,664709.0,321368864,0.78,12.49,8.15,3.86
8,78,id,Indonesia,1904569.0,1811569.0,93000.0,255993674,0.92,16.72,6.37,1.16
9,40,cg,"Congo, Democratic Republic of the",2344858.0,2267048.0,77810.0,79375136,2.45,34.88,10.07,0.27


# 20 Countries have a higher death rate than birth rate

In [11]:
q10= """select * 
from facts 
where birth_rate != 0 and death_rate != 0 and death_rate > birth_rate
order by birth_rate / death_rate ASC"""
pd.read_sql_query(q10,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
1,153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.00
2,96,lg,Latvia,64589,62249,2340,1986705,1.06,10.00,14.31,6.26
3,102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.10,14.27,6.27
4,75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
5,117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
6,158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37
7,183,up,Ukraine,603550,579330,24220,44429471,0.60,10.72,14.46,2.25
8,65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
9,214,sb,Saint Pierre and Miquelon,242,242,0,5657,1.08,7.42,9.72,8.49
